In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import connections
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
import numpy as np
import pandas as pd

# pd.set_option('display.max_rows', 1000)

In [2]:
# Instantiate connection to elasticsearch
es = Elasticsearch(['localhost:9200'])

# Pull userale logs from es
s = Search(using=es, index="userale")
ale_list = []
for hit in s.scan():
    ale_list.append(hit.to_dict())
    
# Convert to pandas df
ale_df = pd.json_normalize(ale_list, sep="_")

# Inspect fields
ale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072 entries, 0 to 1071
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   clientTime       1037 non-null   float64
 1   pageTitle        885 non-null    object 
 2   toolVersion      1037 non-null   object 
 3   microTime        885 non-null    float64
 4   @timestamp       1072 non-null   object 
 5   sessionID        1037 non-null   object 
 6   major_ver        1037 non-null   object 
 7   logType          1037 non-null   object 
 8   userAction       1037 non-null   object 
 9   toolName         1037 non-null   object 
 10  minor_ver        1037 non-null   object 
 11  host             1072 non-null   object 
 12  userId           1037 non-null   object 
 13  pageReferrer     885 non-null    object 
 14  patch_ver        1037 non-null   object 
 15  type             1037 non-null   object 
 16  path             1037 non-null   object 
 17  details       

In [3]:
# Parse View time
is_view = ale_df["type"] == 'view'
view_df = ale_df[is_view][['postId', 'viewTime', 'userId']]
view_df = view_df.astype({"viewTime": int, "postId": int ,"userId": int})
agg_view_time = view_df.groupby(['postId', 'userId'])[['viewTime']].agg('sum')
agg_view_time.reset_index(inplace=True)
agg_view_time['viewTime (s)'] = agg_view_time['viewTime'].div(1000)
agg_view_time

,postId,userId,viewTime,viewTime (s)
0,32,2,149586,149.586
1,35,1,24236,24.236
2,35,2,162353,162.353
3,36,2,156520,156.520
4,37,2,153702,153.702
...,...,...,...,...
88,99,2,202596,202.596
89,100,1,1611787,1611.787
90,100,2,215981,215.981
91,101,2,211622,211.622


In [4]:
# Parse mouseovers
is_mouseover = ale_df['type'] == 'mouseover'
is_user = ale_df['userId'] != 'None'
mouseover_df = ale_df[is_mouseover & is_user][['clientTime', '@timestamp', 'isPost', 'postIds', 'userId']]
mouseover_df.sort_values('clientTime', inplace=True)
mouseover_df = mouseover_df.explode('postIds')
mouseover_df

,clientTime,@timestamp,isPost,postIds,userId
41,1.637713e+12,2021-11-24T00:23:59.256Z,False,NaN,1
36,1.637713e+12,2021-11-24T00:23:59.256Z,False,NaN,1
35,1.637713e+12,2021-11-24T00:23:59.256Z,False,NaN,1
34,1.637713e+12,2021-11-24T00:23:59.256Z,False,NaN,1
52,1.637717e+12,2021-11-24T01:23:34.522Z,False,NaN,1
...,...,...,...,...,...
979,1.637786e+12,2021-11-24T20:35:47.527Z,True,36,2
972,1.637786e+12,2021-11-24T20:35:47.527Z,True,36,2
978,1.637786e+12,2021-11-24T20:35:47.527Z,True,36,2
1062,1.637786e+12,2021-11-24T20:36:22.841Z,False,NaN,2


In [5]:
# group by users
user_mouseover_dfs = []
for _, v in mouseover_df.groupby('userId', as_index=False):
    user_mouseover_dfs.append(v)
user_mouseover_dfs

[       clientTime                @timestamp isPost postIds userId
 41   1.637713e+12  2021-11-24T00:23:59.256Z  False     NaN      1
 36   1.637713e+12  2021-11-24T00:23:59.256Z  False     NaN      1
 35   1.637713e+12  2021-11-24T00:23:59.256Z  False     NaN      1
 34   1.637713e+12  2021-11-24T00:23:59.256Z  False     NaN      1
 52   1.637717e+12  2021-11-24T01:23:34.522Z  False     NaN      1
 ..            ...                       ...    ...     ...    ...
 294  1.637717e+12  2021-11-24T01:29:22.140Z   True      88      1
 791  1.637717e+12  2021-11-24T01:29:22.140Z  False     NaN      1
 286  1.637717e+12  2021-11-24T01:29:22.141Z   True      87      1
 287  1.637717e+12  2021-11-24T01:29:22.141Z   True      87      1
 792  1.637717e+12  2021-11-24T01:29:22.141Z  False     NaN      1
 
 [520 rows x 5 columns],
         clientTime                @timestamp isPost postIds userId
 898   1.637784e+12  2021-11-24T19:54:33.039Z   True      94      2
 908   1.637784e+12  2021-11-24T1

In [6]:
# Calculate mouseover times
def calc_mouseover_times(df):
    # Create index for each mouseover segment
    df['shifted_neq'] = (df['postIds'].shift() != df['postIds'])
    df['groups'] = df['shifted_neq'].cumsum()
    
    # Find diff between start and end times of each segment
    min_max_times = df.groupby(['groups']).agg(
        min_time=pd.NamedAgg(column='clientTime', aggfunc='min'),
        max_time=pd.NamedAgg(column='clientTime', aggfunc='max'),
    )
    min_max_times['diff'] = min_max_times['max_time'] - min_max_times['min_time']
    
    # Re-add postIds
    min_max_times['postId'] = df.groupby('groups')['postIds'].unique().explode()
    min_max_times = min_max_times.groupby('postId')['diff'].sum().divide(1000).reset_index()
    min_max_times.rename(columns={'diff': 'mouseoverTime (s)'}, inplace=True)
    min_max_times['userId'] = df['userId'].unique()[0]
    
    return min_max_times

agg_mouseover_time = pd.concat(list(map(calc_mouseover_times, user_mouseover_dfs)))

In [7]:
# Fix dataframe col-types
agg_mouseover_time = agg_mouseover_time.astype({"postId": int ,"userId": int})
agg_view_time = agg_view_time.astype({"postId": int ,"userId": int})

# Merge view time 
user_post_interactions = pd.merge(agg_mouseover_time, agg_view_time, on=['postId', 'userId'])
user_post_interactions = user_post_interactions[['userId', 'postId', 'mouseoverTime (s)', 'viewTime (s)']]

In [8]:
user_post_interactions

,userId,postId,mouseoverTime (s),viewTime (s)
0,1,100,0.000,1611.787
1,1,102,0.567,804.785
2,1,35,21.263,24.236
3,1,39,1.283,24.684
4,1,41,0.248,23.901
...,...,...,...,...
65,2,92,6.952,223.104
66,2,93,0.200,172.904
67,2,94,3.967,198.288
68,2,96,1.320,220.516
